<a href="https://colab.research.google.com/github/alessioborgi/StyleAlignedDiffModels/blob/main/MetricsStyleAlign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SINGLE-STYLE METRICS

In [1]:
# Download the file
!gdown --folder https://drive.google.com/drive/folders/1Rdb3XkwW1H_IMFVh3tShj4adh-XUNt03?usp=sharing

zsh:1: no matches found: https://drive.google.com/drive/folders/1Rdb3XkwW1H_IMFVh3tShj4adh-XUNt03?usp=sharing


In [2]:
# Unzip the file if it's a zip
!unzip -q AlignZipFolder/OnlyPromptFile.zip

unzip:  cannot find or open AlignZipFolder/OnlyPromptFile.zip, AlignZipFolder/OnlyPromptFile.zip.zip or AlignZipFolder/OnlyPromptFile.zip.ZIP.


In [3]:
# Install required libraries
!pip install transformers ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Using cached wcwidth-0.2.13-py2.py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 983.3 kB/s eta 0:00:000:00:01
Using cached wcwidth-0.2.13-py2.py3-none-any.whl (34 kB)
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/mx/xlmhp5ln7l5g33gb6w32s46m0000gn/T/pip-req-build-e12_6icu
  Running command git clone --quiet https://github.com/openai/CLIP.git /private/var/folders/mx/xlmhp5ln7l5g33gb6w32s46m0000gn/T/pip-req-build-e12_6icu
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369520 sha256=04371c464313409990a7b4e

In [4]:
import os
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import AutoImageProcessor, AutoModel
from torchvision import transforms

# Load CLIP model and preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# Load DINO model and processor from Hugging Face
dino_processor = AutoImageProcessor.from_pretrained("facebook/dino-vitb8")
dino_model = AutoModel.from_pretrained("facebook/dino-vitb8").to(device)

def get_clip_similarity(image_path, prompt):
    image = clip_preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text = clip.tokenize([prompt]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        text_features = clip_model.encode_text(text)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity = (image_features @ text_features.T).item()
    return similarity

def get_dino_embeddings(image_paths):
    images = [Image.open(image_path) for image_path in image_paths]
    inputs = dino_processor(images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Taking the CLS token

    return embeddings

def compute_pairwise_similarity(embeddings):
    similarities = cosine_similarity(embeddings)
    upper_tri_indices = np.triu_indices_from(similarities, k=1)
    return similarities[upper_tri_indices].mean()

def process_folder(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    clip_results = {'aligned': [], 'non_aligned': []}
    dino_results = {'aligned': [], 'non_aligned': []}

    for subfolder in subfolders:
        aligned_images = []
        non_aligned_images = []
        subfolder_name = os.path.basename(subfolder)

        for file in os.listdir(subfolder):
            if file.endswith(".jpg") or file.endswith(".png"):
                file_path = os.path.join(subfolder, file)
                if file.startswith("znon"):
                    non_aligned_images.append(file_path)
                else:
                    aligned_images.append(file_path)

        for image_path in aligned_images:
            image_name = os.path.basename(image_path)
            prompt = f"{image_name[:-4]}, {subfolder_name}"
            clip_similarity = get_clip_similarity(image_path, prompt)
            clip_results['aligned'].append(clip_similarity)

        for image_path in non_aligned_images:
            image_name = os.path.basename(image_path)[5:]
            prompt = f"{image_name[:-4]}, {subfolder_name}"
            clip_similarity = get_clip_similarity(image_path, prompt)
            clip_results['non_aligned'].append(clip_similarity)

        if aligned_images:
            aligned_embeddings = get_dino_embeddings(aligned_images)
            dino_similarity = compute_pairwise_similarity(aligned_embeddings)
            dino_results['aligned'].append(dino_similarity)

        if non_aligned_images:
            non_aligned_embeddings = get_dino_embeddings(non_aligned_images)
            dino_similarity = compute_pairwise_similarity(non_aligned_embeddings)
            dino_results['non_aligned'].append(dino_similarity)

    avg_clip_aligned = np.mean(clip_results['aligned'])
    avg_clip_non_aligned = np.mean(clip_results['non_aligned'])
    avg_dino_aligned = np.mean(dino_results['aligned'])
    avg_dino_non_aligned = np.mean(dino_results['non_aligned'])

    return {
        "clip_aligned": avg_clip_aligned,
        "clip_non_aligned": avg_clip_non_aligned,
        "dino_aligned": avg_dino_aligned,
        "dino_non_aligned": avg_dino_non_aligned
    }

# Example usage
folder_path = "OnlyPrompt"
results = process_folder(folder_path)
print(results)

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 61.8MiB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'clip_aligned': 0.34217529296875, 'clip_non_aligned': 0.3454345703125, 'dino_aligned': 0.5018175, 'dino_non_aligned': 0.34539917}


In [5]:
results

{'clip_aligned': 0.34217529296875,
 'clip_non_aligned': 0.3454345703125,
 'dino_aligned': 0.5018175,
 'dino_non_aligned': 0.34539917}

### MULTI-STYLE DINO VIT-B/8

In [ ]:
!git clone https://github.com/alessioborgi/StyleAlignedDiffModels.git

# Change directory to the cloned repository
%cd StyleAlignedDiffModels
%ls

In [ ]:
# Download the file
!gdown --folder https://drive.google.com/drive/folders/1Rdb3XkwW1H_IMFVh3tShj4adh-XUNt03?usp=sharing

In [ ]:
# Unzip the file if it's a zip
!unzip -q AlignZipFolder/OnlyPromptFile.zip

In [ ]:
# Install required libraries
!pip install transformers ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [10]:
import numpy as np
import os
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load DINO model and processor from Hugging Face
device = "cuda" if torch.cuda.is_available() else "cpu"
dino_processor = AutoImageProcessor.from_pretrained("facebook/dino-vitb8")
dino_model = AutoModel.from_pretrained("facebook/dino-vitb8").to(device)


def get_dino_embeddings(image_paths):
    """
    Extract DINO VIT-B/8 embeddings from a list of image paths.
    
    :param image_paths: List of paths to images.
    :return: List of embeddings corresponding to each image.
    """
    images = [Image.open(image_path) for image_path in image_paths]
    inputs = dino_processor(images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Taking the CLS token

    return embeddings

def compute_weighted_similarity(generated_embedding, reference_embeddings, weights):
    """
    Compute the weighted similarity between the generated image and reference styles.
    
    :param generated_embedding: Embedding of the generated image (1D numpy array).
    :param reference_embeddings: List of embeddings for the reference images (list of 1D numpy arrays).
    :param weights: List of weights corresponding to each reference image.
    :return: Weighted similarity score.
    """
    similarities = []
    for ref_embedding in reference_embeddings:
        similarity = cosine_similarity(generated_embedding.reshape(1, -1), ref_embedding.reshape(1, -1)).item()
        similarities.append(similarity)

    weighted_similarity = np.dot(similarities, weights) / sum(weights)
    return weighted_similarity

def calculate_weighted_multi_style_dino(image_path, reference_image_paths, weights):
    """
    Calculate the Weighted Multi-Style DINO VIT-B/8 metric.
    
    :param image_path: Path to the generated image.
    :param reference_image_paths: List of paths to the reference style images.
    :param weights: List of weights corresponding to each reference image.
    :return: Weighted Multi-Style DINO VIT-B/8 metric value.
    """
    # Get the embedding for the generated image
    generated_embedding = get_dino_embeddings([image_path])[0]

    # Get the embeddings for the reference images
    reference_embeddings = get_dino_embeddings(reference_image_paths)

    # Compute the weighted similarity
    weighted_similarity = compute_weighted_similarity(generated_embedding, reference_embeddings, weights)

    return weighted_similarity

def process_main_folder(main_folder, reference_image_paths):
    """
    Process the main folder and compute the weighted DINO metric for each subfolder.
    """
    results = {}

    subfolders = [f.path for f in os.scandir(main_folder) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        # Extract weights from the subfolder name
        try:
            parts = subfolder_name.split('_')
            weight1 = float(parts[-2])
            weight2 = float(parts[-1])
            weights = [weight1, weight2]
        except Exception as e:
            print(f"Error parsing weights from {subfolder_name}: {e}")
            continue

        subfolder_results = []

        for image_file in os.listdir(subfolder):
            if image_file.endswith(".jpg") or image_file.endswith(".png"):
                image_path = os.path.join(subfolder, image_file)
                # Calculate the weighted DINO metric
                weighted_dino_metric = calculate_weighted_multi_style_dino(image_path, reference_image_paths, weights)
                subfolder_results.append((image_file, weighted_dino_metric))

        # Store results by subfolder
        results[subfolder_name] = subfolder_results

    return results


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


: 

In [ ]:
main_folder = "./imgs/Multi_Reference/Weighted_Latent_Space/medieval_picasso_Guidance_10"
reference_image_paths = ["./imgs/medieval-bed.jpeg", "./imgs/Picasso_Smoking_Water_Pipe.jpeg"]
results = process_main_folder(main_folder, reference_image_paths)

# Print or save the results
for subfolder, subfolder_results in results.items():
    print(f"Results for {subfolder}:")
    for image_file, metric in subfolder_results:
        print(f"Image: {image_file}, Weighted DINO Metric: {metric}")